In [593]:
import os
import pandas as pd

In [594]:
def get_ID_and_loc(file_path):
    pathsplit='_'.join(file_path.split('/')[1:]).split('_')
    return [pathsplit[-2], pathsplit[-1]]

In [595]:
def threeT_vs_15T(table_file_path):
    df=pd.read_csv(table_file_path, error_bad_lines=False, skiprows=2, nrows=1, header=None, index_col=False)
#     print(df.iloc[0,0])
    if df.iloc[0, 0] == ' (Huntington MR Center)':
        return "1.5T"
    else:
        return "3T"

In [596]:
def get_DOE(table_file_path):
    df=pd.read_csv(table_file_path, sep=' +', engine='python', error_bad_lines=False, skiprows=1, nrows=1, header=None, index_col=False)
    return df.iloc[0, 3]

In [597]:
def table_to_row(df):
    df_out = df.stack()
    df_out.index = df_out.index.map('{0[0]}{0[1]}'.format)
    return df_out.to_frame().T

In [598]:
def read_table_file(dirpath, table_file_path):
    #get ID and sample location from path
    ID=get_ID_and_loc(dirpath)[0]
    tissue=get_ID_and_loc(dirpath)[1]
    
    #get scanner info
    scanner_T=threeT_vs_15T(table_file_path)
    
    #get Date of Exam
    DOE=get_DOE(table_file_path)
    
    #get data 
    data=pd.read_csv(table_file_path,sep=' +', engine='python',error_bad_lines=False, skiprows=7, nrows=36)
    data=data.set_index('Metabolite')
    data=table_to_row(data)
    
    data['Scanner']=scanner_T
    data['ID']=ID
    data['Tissue']=tissue
    data['DOE']=DOE
    data=data.set_index('ID')
    return data

In [615]:
def main ():
    dir='/Users/baymac/Desktop/BR samples'
    
    pattern='table'
    exten='.csv'
    
    data_final=pd.DataFrame()
    
    #read MRS contents
    for dirpath, subdirs, files in os.walk(dir):
        for file in files:
            if (pattern in file) and (len(file)==5):
                #changae file format to csv
                src=os.path.join(dirpath, file)
                dst=os.path.join(dirpath, file+'.csv')
                os.rename(src,dst)

                #read file now
                data=read_table_file(dirpath, dst)
                data_final=data_final.append(data)
            elif (pattern in file) and file.endswith('.csv'):
                data=read_table_file(dirpath, os.path.join(dirpath,file))
                data_final=data_final.append(data)
                
#     print(data_final)
    data_final.to_csv("/Users/baymac/Desktop/MRS_test.csv")
    return

In [616]:
if __name__ == '__main__': main()